* It is also known as map side as well as replicated join.
* The smaller data set will be broadcasted to all the executors in the cluster.
* The size of the smaller data set is driven by `spark.sql.autoBroadcastJoinThreshold`.
* We can even perform broadcast join when the smaller data set size is greater than `spark.sql.autoBroadcastJoinThreshold` by using `broadcast` function from `pyspark.sql.functions`.
* We can disable broadcast join by setting `spark.sql.autoBroadcastJoinThreshold` value to 0.
* If broadcast join is disabled then it will result in reduce side join.
* Make sure to setup multinode cluster using 28 GB Memory, 4 Cores each. Configure scaling between 2 and 4 nodes. Driver can be of minimum configuration.

In [0]:
# Default size is 10 MB.
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

Out[1]: '10485760b'

In [0]:
# We can disable broadcast join using this approach
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '0')

In [0]:
# Resetting to original value
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '10485760b')

In [0]:
# 1+ GB Data Set
clickstream = spark.read.csv('dbfs:/databricks-datasets/wikipedia-datasets/data-001/clickstream/raw-uncompressed/', sep='\t', header=True)

In [0]:
# 10+ GB Data Set
articles = spark.read.parquet('dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet/')

In [0]:
%%time

# Default will be reduce side join as the size of smaller data set is more than 10 MB (default broadcast size)
clickstream.join(articles, articles.id == clickstream.curr_id).count()

CPU times: user 720 ms, sys: 294 ms, total: 1.01 s
Wall time: 4min
Out[6]: 19959125

* Review SQL Plan visualization to confirm the previous query have used sort merge join.

In [0]:
from pyspark.sql.functions import broadcast

In [0]:
%%time
# We can use broadcast function to override existing broadcast join threshold
# We can also override by using this code spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '1500m')
broadcast(clickstream).join(articles, articles.id == clickstream.curr_id).count()

CPU times: user 691 ms, sys: 342 ms, total: 1.03 s
Wall time: 4min
Out[8]: 19959125

* Review SQL Plan visualization to confirm the previous query have used broadcast based join.